Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give editor access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [1]:
## DONT CHANGE THIS CELL 
# this is currently the same as dev.data.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt

--2021-10-25 12:27:08--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt [following]
--2021-10-25 12:27:09--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63639 (62K) [text/plain]
Saving to: ‘test.data.txt’

test.data.txt       100%[===================>]  62.15K   130KB/s    in 0.5s    

2021-10-25 12:27:10 (130 KB/s) - ‘test.data.txt’ saved [63639/63639]



In [2]:
## Replace with the right link that contains the zip file uploaded from the training
# !gdown https://drive.google.com/uc?id=xxxxxxxxxxx
!gdown https://drive.google.com/uc?id=1lYDa3t45CuLHZgmubujGkqgTd38UdNAZ

Downloading...
From: https://drive.google.com/uc?id=1lYDa3t45CuLHZgmubujGkqgTd38UdNAZ
To: /content/2021CSY7548_B_model.zip
100% 43.4M/43.4M [00:01<00:00, 37.2MB/s]


In [3]:
!pip install sentencepiece
!pip install transformers

     |████████████████████████████████| 1.2 MB 13.0 MB/s 
     |████████████████████████████████| 2.9 MB 12.8 MB/s 
     |████████████████████████████████| 596 kB 41.9 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 3.3 MB 42.1 MB/s 
     |████████████████████████████████| 895 kB 45.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
## YOUR testing code
from transformers import AlbertConfig, AlbertModel, AlbertTokenizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AlbertForSequenceClassification, AdamW
import pandas as pd
import numpy as np
import pickle
import torch
from torch.utils.data import TensorDataset, random_split

In [8]:
device = "cpu"

if torch.cuda.is_available():
    device = "cuda"

device

'cuda'

In [9]:
!unzip /content/2021CSY7548_B_model.zip

Archive:  /content/2021CSY7548_B_model.zip
   creating: content/model/
  inflating: content/model/weight_params.pt  


In [10]:
with open("/content/test.data.txt","r") as f:
  test_data = f.read()

c=0
dataset = []
temp = {"sent1":"","sent2":"","word":"","pos1":0,"pos2":0}
for data in test_data.split("\n"):
  if(data != ''):
      temp = {"sent1":"","sent2":"","word":"","pos1":0,"pos2":0,"label":0}
      temp["text"] = data.split("\t")[3] + " [SEP] " + data.split("\t")[4]
      temp["sent1"] = data.split("\t")[3]
      temp["sent2"] = data.split("\t")[4]
      temp["pos1"] = int(data.split("\t")[2].split("-")[0])
      temp["pos2"] = int(data.split("\t")[2].split("-")[1])
      temp["word"] = data.split("\t")[0]
      temp["pos_tag"] = data.split("\t")[1]
      dataset.append(temp)
      c += 1

pd.DataFrame(dataset).to_csv("/content/test_data.tsv",sep="\t",index=False)

In [14]:
model_name = "albert-base-v2"
model = AlbertForSequenceClassification.from_pretrained(model_name, num_labels = 2)

path='content/model/weight_params.pt'
model.load_state_dict(torch.load(path,map_location=torch.device(device))["model_dict"]);
model.eval();

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.decoder.weight', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [15]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

In [16]:
def prepare_input_data(all_data):
    
    tokenized_text = []
    attention_masks = []
    labels = []

    for sentence in all_data["text"].tolist():
        encoded_dict = tokenizer.encode_plus(sentence, add_special_tokens = True, max_length = 64, pad_to_max_length = True,\
                                             return_attention_mask = True, return_tensors = 'pt')

        tokenized_text.append(encoded_dict['input_ids'])    
        attention_masks.append(encoded_dict['attention_mask'])

    tokenized_text = torch.cat(tokenized_text, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    
    return tokenized_text, attention_masks

test_data = pd.read_csv("/content/test_data.tsv",sep="\t")
test_tokenized_text, test_attention_masks = prepare_input_data(test_data)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [17]:
batch_size = 32
test_dataset = TensorDataset(test_tokenized_text, test_attention_masks)
test_dataloader = DataLoader(test_dataset, sampler = SequentialSampler(test_dataset), batch_size = batch_size )

In [18]:
model.eval()

predictions = []

for batch in test_dataloader:

  tokenized_text_data, att_mask = batch
  tokenized_text_data.to(device)
  att_mask.to(device)
  
  with torch.no_grad():
      outputs = model(tokenized_text_data, attention_mask=att_mask)

  logits = outputs[0]
  logits = logits.detach().cpu().numpy()  
  pred_flat = np.argmax(logits, axis=1).flatten()
  predictions.extend(pred_flat)


In [19]:
fout = open("output.txt", "w")
for label in predictions:
  if(label == 1):
    fout.write("T\n")
  else:
    fout.write("F\n")
fout.close()

In [20]:
## DONT CHANGE THIS CELL
# Your testing code must produce a file output.txt with predictions as T and F in each line

## Final Evaluation 
# this is currently the same as dev.gold.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
correct, total = 0., 0.
for prediction, gold in zip(open('output.txt'), open('test.gold.txt')):
  prediction, gold = prediction.strip(), gold.strip()
  total += 1
  if prediction == gold:
    correct += 1

## Report this as the final validation performance 
print('Performance = ', (correct/total))

--2021-10-25 12:42:58--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt [following]
--2021-10-25 12:42:59--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1276 (1.2K) [text/plain]
Saving to: ‘test.gold.txt’

test.gold.txt       100%[===================>]   1.25K  --.-KB/s    in 0s      

2021-10-25 12:42:59 (59.5 MB/s) - ‘test.gold.txt’ saved [1276/1276]

Performance =  0.7586206896551724
